# Model Tuning with Grid Search

## Objectives

- Explain what hyperparameters are
- Describe the purpose of grid searching
- Implement grid searching for the purposes of model optimization

![](https://imgs.xkcd.com/comics/machine_learning.png)

## Hyperparameters

Many of the models we have looked at are really *families* of models in the sense that they make use of **hyperparameters**.

Thus for example the $k$-nearest-neighbors algorithm allows us to make:

- a 1-nearest-neighbor model
- a 2-nearest-neighbors model
- a 3-nearest-neighbors model
- etc.

Or, for another example, the decision tree algorithm allows us to make:

- a classifier that branches according to information gain
- a classifier that branches according to Gini impurity
- a regressor that branches according to mean squared error
- etc.

Depending on the sort of problem and data at hand, it is natural to experiment with different values of these hyperparameters to try to improve model performance.

> We can think of these **hyperparameters** as _dials_ of the base model

<img src='https://cdn.dribbble.com/users/947489/screenshots/4522230/0001-0128.gif' width=500>

### Difference from Parametric / Non-Parametric Models

Contrast the notion of hyperparameters with the distinction between **parametric** and **non-parametric** models.

A linear regression model is parametric in the sense that we start with a given model *form* and we then search for the optimal parameters to fill in that form. But *those* parameters are not the sort we might tweak for the purposes of improving model performance! On the contrary, there is **one** best set of parameters, and the training of the model is a matter of finding those optimal values.

## Scenario: Cat in the Dat(a)

Let's revisit our cat in the data kaggle competition: https://www.kaggle.com/c/cat-in-the-dat-ii

In [1]:
# Imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import roc_auc_score, plot_roc_curve, plot_precision_recall_curve
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

In [2]:
# Grab, then explore data
df = pd.read_csv("data/cat_in_the_dat2_train.csv", index_col='id')

In [3]:
# Define our X and y

X = df.drop(columns=['target'])
y = df['target']

# and train test split - to create our val holdout set!
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1,
                                                  random_state=0)

### Preprocessor Pipeline

Let's bring back our preprocessor pipeline we built:

In [4]:
# First defining our overall used columns - those with less than 30 uniques
used_cols = [col for col in X_train.columns if len(X_train[col].unique()) < 30]

# Defining our object columns
obj_cols = [col for col in used_cols if X_train[col].dtype == 'O']
# And our object transformer
obj_transformer = Pipeline(steps=[
    ("obj_imputer", SimpleImputer(strategy = 'constant', fill_value='WHATEVER')),
    ('ohe', OneHotEncoder(drop=['WHATEVER']*len(obj_cols)))
])

# Now defining our numeric columns
num_cols = [col for col in used_cols if X_train[col].dtype != 'O']
# And our numeric transformer
num_transformer = Pipeline(steps = [
    ('num_imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', MinMaxScaler())
])

# Putting them together into a preprocessor
preprocessor = ColumnTransformer(transformers=[
    ("num_trans", num_transformer, num_cols),
    ('obj_trans', obj_transformer, obj_cols)
])

In [9]:
num_cols

['bin_0', 'bin_1', 'bin_2', 'ord_0', 'day', 'month']

In [8]:
ohe_col_names = preprocessor.named_transformers_['obj_trans']['ohe'].get_feature_names()

In [10]:
output_cols = [*num_cols, *ohe_col_names]
len(output_cols)

87

In [15]:
pd.DataFrame(preprocessor.fit_transform(X_train[used_cols]).A,
             index=X_train.index, columns=output_cols).head()

,bin_0,bin_1,bin_2,ord_0,day,month,x0_F,x0_T,x1_N,x1_Y,...,x10_Q,x10_R,x10_S,x10_T,x10_U,x10_V,x10_W,x10_X,x10_Y,x10_Z
id,,,,,,,,,,,,,,,,,,,,,
111903,0.0,0.0,0.0,1.0,0.500000,0.727273,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
521435,0.0,1.0,0.0,0.5,0.666667,0.000000,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
484488,0.0,0.0,0.0,0.5,0.166667,0.636364,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
529639,0.0,1.0,1.0,0.0,0.666667,0.090909,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
589943,0.0,0.0,0.0,0.5,0.833333,0.636364,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Trying Different Models & Values

Let's explore three different model types, using our preprocessor pipeline and evaluating using `cross_validate`:

#### Logistic Regression

In [17]:
clf_lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('logreg', LogisticRegression(random_state=713))
])

In [18]:
cv_results = cross_validate(clf_lr, X_train[used_cols], y_train,
                            scoring=['roc_auc', 'f1'])

/Users/litzysalazar/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/litzysalazar/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-lear

In [19]:
cv_resultsts_results_results

{'fit_time': array([5.44178009, 4.98794913, 5.40342498, 5.46978402, 5.36496735]),
 'score_time': array([1.23542595, 1.21480918, 1.21470714, 1.21830416, 1.20083475]),
 'test_roc_auc': array([0.73592695, 0.73430216, 0.73870237, 0.73259681, 0.73553261]),
 'test_f1': array([0.16649166, 0.17204481, 0.16301652, 0.16921013, 0.17259222])}

#### Decision Tree

In [20]:
clf_lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('tree', DecisionTreeClassifier(max_depth=7, random_state=713))
])

In [21]:
cv_results = cross_validate(clf_lr, X_train[used_cols], y_train,
                            scoring=['roc_auc', 'f1'])
cv_results

{'fit_time': array([4.17166996, 4.09052014, 4.06305385, 4.10969496, 4.07737374]),
 'score_time': array([1.21341419, 1.20721102, 1.22462106, 1.21202898, 1.20578599]),
 'test_roc_auc': array([0.67141229, 0.66792007, 0.67202339, 0.66846962, 0.66986597]),
 'test_f1': array([0.05099737, 0.04137211, 0.04184697, 0.03716586, 0.05640116])}

#### Your Choice: KNN or Naive Bayes?

In [22]:
clf_nb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('bayes', MultinomialNB())
])

In [23]:
cv_results = cross_validate(clf_lr, X_train[used_cols], y_train,
                            scoring=['roc_auc', 'f1'])
cv_results

{'fit_time': array([4.14700794, 4.07600212, 4.03238201, 4.04965591, 4.0329659 ]),
 'score_time': array([1.2076838 , 1.20068669, 1.20609283, 1.19835114, 1.20371914]),
 'test_roc_auc': array([0.67141229, 0.66792007, 0.67202339, 0.66846962, 0.66986597]),
 'test_f1': array([0.05099737, 0.04137211, 0.04184697, 0.03716586, 0.05640116])}

## Automatically Searching for Optimal Hyperparameters with Grid Search

It's not a bad idea to experiment with the values of your models' hyperparameters a bit as you're getting a feel for your models' performance. But there are more systematic ways of going about the search for optimal hyperparameters. One method of hyperparameter tuning is **grid searching**. 

The idea is to build multiple models with different hyperparameter values and then see which one performs the best. The hyperparameters and the values to try form a sort of *grid* along which we are looking for the best performance. 

For example, for KNN:

    n_neighbors | metric      | weights
    ______________________________________
    1           | 'minkowski' | 'uniform'
    3           | 'manhattan' | 'distance'
    5           |


Scikit-Learn has a [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) class whose `fit()` method runs this procedure. Note that this can be quite computationally expensive since:

- A model is constructed for each combination of hyperparameter values that we input; and
- Each model is cross-validated.

### `GridSearchCV`

Note! When using `GridSearchCV` in SKLearn with a pipeline, you need to use the nicknames we give to tell it where these hyperparameters are being tested!

Resource: https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html

In [24]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

In [25]:
# Choose one model type to optimize, and set up the base pipeline

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
     ('tree', DecisionTreeClassifier(max_depth=7, random_state=713))
])

In [28]:
# Define the parameter grid for one model type

param_grid = {
    "tree__max_depth": [5,14,22,81],
    "tree__min_samples_leaf":[5,14,22]
}

**Question: How many models will we be constructing with this grid?**

- 60(4*3) with cv=5

In [30]:
# Initialize the grid search object with five-fold cross-validation

gs = GridSearchCV(clf, param_grid, cv=5, scoring='roc_auc')

# Then fit it to our X_train and y_train
gs.fit(X_train[used_cols], y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num_trans',
                                                                         Pipeline(steps=[('num_imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['bin_0',
                                                                          'bin_1',
                                                                          'bin_2',
                                                                          'ord_0',
                                                             

In [31]:
# Check out the best parameters
gs.best_params_

{'tree__max_depth': 14, 'tree__min_samples_leaf': 22}

In [32]:
# Check out the best score
gs.best_score_

0.6805734653573501

In [33]:
# We can grab the best estimator out, as a fit model!
gs.best_estimator_

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_trans',
                                                  Pipeline(steps=[('num_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['bin_0', 'bin_1', 'bin_2',
                                                   'ord_0', 'day', 'month']),
                                                 ('obj_trans',
                                                  Pipeline(steps=[('obj_imputer',
                                                                   SimpleImputer(fill_value='WHATEVER',
                                                                                 strategy='constant')),
                                         

In [34]:
# A bigger picture of our results
gs.cv_results_

{'mean_fit_time': array([ 3.25591412,  3.22279935,  3.24336448, 15.32972002, 15.31376567,
        14.93602076, 36.40502176, 50.27467561, 30.06150694, 45.3831984 ,
        37.04885125, 33.75754194]),
 'std_fit_time': array([6.27779439e-02, 1.36968860e-02, 2.32207504e-02, 1.62874816e-01,
        8.43496103e-01, 2.49216982e-01, 2.97385813e-01, 3.64337740e+01,
        3.16255283e-01, 3.77894098e-01, 4.30575786e-01, 1.34050638e-01]),
 'mean_score_time': array([0.58812785, 0.58377819, 0.59238472, 0.60441079, 0.61725693,
        0.60219927, 0.61615419, 0.63565316, 0.62558665, 0.61247072,
        0.62455225, 0.62075624]),
 'std_score_time': array([0.00934681, 0.00133394, 0.00641686, 0.00675473, 0.03967682,
        0.00529775, 0.01370401, 0.04423979, 0.01375402, 0.00737139,
        0.01560415, 0.00597591]),
 'param_tree__max_depth': masked_array(data=[5, 5, 5, 14, 14, 14, 22, 22, 22, 81, 81, 81],
              mask=[False, False, False, False, False, False, False, False,
                    Fal

In [35]:
# Which we can throw into a dataframe if we like
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_tree__max_depth,param_tree__min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,3.255914,0.062778,0.588128,0.009347,5,5,"{'tree__max_depth': 5, 'tree__min_samples_leaf...",0.646981,0.646451,0.647599,0.646708,0.646017,0.646751,0.000529,8
1,3.222799,0.013697,0.583778,0.001334,5,14,"{'tree__max_depth': 5, 'tree__min_samples_leaf...",0.646981,0.646451,0.647599,0.646708,0.646018,0.646751,0.000529,6
2,3.243364,0.023221,0.592385,0.006417,5,22,"{'tree__max_depth': 5, 'tree__min_samples_leaf...",0.646981,0.646451,0.647599,0.646708,0.646018,0.646751,0.000529,6
3,15.329720,0.162875,0.604411,0.006755,14,5,"{'tree__max_depth': 14, 'tree__min_samples_lea...",0.671365,0.669342,0.673427,0.669577,0.672600,0.671262,0.001613,3
4,15.313766,0.843496,0.617257,0.039677,14,14,"{'tree__max_depth': 14, 'tree__min_samples_lea...",0.677498,0.674446,0.677906,0.674934,0.679177,0.676792,0.001810,2
5,14.936021,0.249217,0.602199,0.005298,14,22,"{'tree__max_depth': 14, 'tree__min_samples_lea...",0.681744,0.679205,0.681055,0.678374,0.682489,0.680573,0.001548,1
6,36.405022,0.297386,0.616154,0.013704,22,5,"{'tree__max_depth': 22, 'tree__min_samples_lea...",0.604850,0.603150,0.609878,0.605264,0.603747,0.605378,0.002373,11
7,50.274676,36.433774,0.635653,0.044240,22,14,"{'tree__max_depth': 22, 'tree__min_samples_lea...",0.645081,0.644015,0.646470,0.644455,0.646383,0.645281,0.000995,9
8,30.061507,0.316255,0.625587,0.013754,22,22,"{'tree__max_depth': 22, 'tree__min_samples_lea...",0.661195,0.658876,0.660150,0.656970,0.661909,0.659820,0.001754,4
9,45.383198,0.377894,0.612471,0.007371,81,5,"{'tree__max_depth': 81, 'tree__min_samples_lea...",0.596413,0.597044,0.598408,0.594360,0.596372,0.596519,0.001307,12


### Choice of Grid Values

Which values should you pick for your grid? Intuitively, you should try both "large" and "small" values, but of course what counts as large and small will really depend on the type of hyperparameter.

- For a k-nearest neighbors model, 1 or 3 would be a small value for the number of neighbors and 15 or 17 would be a large value.
- For a decision tree model, what counts as a small `max_depth` will really depend on the size of your training data. A `max_depth` of 5 would likely have little effect on a very small dataset but, at the same time, it would probably significantly decrease the variance of a model where the dataset is large.
- For a logistic regression's regularization constant, you may want to try a set of values that are exponentially separated, like \[1, 10, 100, 1000\].
- **If a grid search finds optimal values at the ends of your hyperparameter ranges, you might try another grid search with more extreme values.**

### Exercise

Do a grid search on a different model type! What are the optimal values for the hyperparameters you've chosen?

In [ ]:
# Example logistic regression param grid, with model nickname'lr'
lr_grid - {
    'lr__C':[.01,.1,1,10,100],
    'lr__penalty':['l1','l2']
}

# Level Up: Random Searching

It is also possible to search for good hyperparameter values randomly. This is a nice choice if computation time is an issue or if you are tuning over continuous hyperparameters.

### `RandomizedSearchCV` with `LogisticRegression`

In [ ]:
# Import
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
log_reg_grid = {'C': stats.uniform(loc=0, scale=10),
               'l1_ratio': stats.expon(scale=0.2)}

In [ ]:
rs = RandomizedSearchCV(estimator=LogisticRegression(penalty='elasticnet',
                                                    solver='saga',
                                                    max_iter=1000,
                                                    random_state=42),
                        param_distributions=log_reg_grid,
                        random_state=42)

rs.fit(X_train, y_train)

rs.best_params_

# Level Up: More Grid Search Practice

Use a classifier of your choice to predict the category of price range for the phones in this dataset. Try tuning some hyperparameters using a grid search, and then write up a short paragraph about your findings.

In [ ]:
phones_train = pd.read_csv('data/phone-train.csv')
phones_test = pd.read_csv('data/phone-test.csv')

In [ ]:
# Your code here